In [ ]:
!pip install azure-ai-ml
!pip install tensorflow

In [3]:
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("setas", version="1")

     |████████████████████████████████| 9.4 MB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 267 kB 88.5 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 4.8 MB/s  eta 0:00:01
     |████████████████████████████████| 110 kB 77.7 MB/s eta 0:00:01
     |████████████████████████████████| 123 kB 90.3 MB/s eta 0:00:01
     |████████████████████████████████| 251 kB 89.6 MB/s eta 0:00:01
ERROR: azure-storage-file-share 12.15.0 has requirement azure-core<2.0.0,>=1.28.0, but you'll have azure-core 1.26.4 which is incompatible.
ERROR: pydash 7.0.7 has requirement typing-extensions!=4.6.0,>=3.10, but you'll have typing-extensions 4.6.0 which is incompatible.
ERROR: azure-storage-file-datalake 12.14.0 has requirement azure-core<2.0.0,>=1.28.0, but you'll have azure-core 1.26.4 which is incompatible.
ERROR: azure-storage-file-datalake 12.14.0 has requirement azure-storage-blob<13.0.0,>=12.19.0, but you'll have azure-storage-blob 12.13.0 which is incompatible.


Found the config file in: /config.json


In [5]:
from azureml.core import Workspace, Datastore, Dataset

# Cargar el workspace desde el archivo de configuración
ws = Workspace.from_config()

# Acceder al datastore por su nombre
datastore_name = 'workspaceblobstore'  # Asegúrate de que este es el nombre correcto de tu datastore
datastore = Datastore.get(ws, datastore_name)

# Crear un Dataset que apunte al archivo ZIP en el datastore
dataset = Dataset.File.from_files((datastore, 'UI/2024-03-18_094615_UTC/Mushrooms.zip'))

# Descargar el archivo ZIP a un directorio local en la máquina virtual de Azure
downloaded_path = dataset.download(target_path='/tmp', overwrite=True)

# El archivo ZIP ahora debería estar en '/tmp' en tu entorno de Azure
print(downloaded_path)


['/tmp/Mushrooms.zip']


In [6]:
import tensorflow as tf
import zipfile
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directorio donde quieres extraer las carpetas y archivos
extract_to = '/tmp/mushrooms'

# Descomprimir el archivo
with zipfile.ZipFile(downloaded_path[0], 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# Ahora, 'extract_to' contiene la estructura de carpetas descomprimida
# La ruta a las carpetas donde se encuentran las imágenes para tu CNN sería algo como:
images_path = os.path.join(extract_to, 'Mushrooms')

# Asegúrate de que 'images_path' sea la ruta correcta a las carpetas de imágenes
# Puedes listar las carpetas para asegurarte
print("Carpetas encontradas:", os.listdir(images_path))

# Configuración de ImageDataGenerator para el entrenamiento
train_datagen = ImageDataGenerator(rescale=1./255)

# Este ejemplo asume que tienes una estructura de directorios con subcarpetas para cada clase
train_generator = train_datagen.flow_from_directory(
    images_path,  # Esto debería ser el camino a la estructura de carpetas
    target_size=(150, 150),  # Tamaño al que se redimensionarán las imágenes
    batch_size=32,
    class_mode='categorical'  # 'categorical' si tienes más de dos clases
)

# Ahora 'train_generator' puede ser usado para alimentar una CNN con las imágenes

Carpetas encontradas: ['Lactarius', 'validation2', '.DS_Store', 'Agaricus', 'Boletus', 'train', 'Russula', 'Suillus', 'Hygrocybe', 'validation', 'train2', 'Amanita', 'Entoloma', 'Cortinarius']
Found 14494 images belonging to 13 classes.
